# Using stochastic gradient descent to fit a line through 2 points 

Yes, I'm aware it might be slightly overkill.

In [1]:
import math
import operator
import random

import plotly.offline as plotly
import plotly.graph_objs as go

plotly.init_notebook_mode(connected=True)

## Data

In [2]:
# Data
X = [7, -3]

# Number of data points
N = len(X)

# Data dimension
d = 1

# Target
Y = [2, 4]

In [3]:
# Our two points
list(zip(X, Y))

[(7, 2), (-3, 4)]

## Model and error definition

In [4]:
# Linear model
def f(w, x):

    # `w` stands for "weights".
    
    # Let w = [a, b]. Return f(x) = a * x + b
    # Hence, f(x) = a * x + b * 1
    # Let z = [x, 1], then f(x) = a * z[0] + b * z[1]
    # Or, f(x) = w[0] * z[0] + w[1] * z[1]

    z = (x, 1)
    return sum(w[j] * z[j] for j in range(d + 1))

# Error committed by the model (relatively to the target) for a given data point
def residual(w, x, y):
    return f(w, x) - y

# Loss function (mean squared error). Error committed by the model for all data points.
def loss(w, X, Y):
    return sum(residual(w, X[i], Y[i]) ** 2 for i in range(N))

## Objective: find w such that it minimizes loss(w, X, Y)

In [6]:
# Gradient of the loss function
def loss_gradient(w, x, y):
    z = (x, 1)
    return [2 * z[j] * residual(w, x, y) for j in range(d + 1)]

# Update rule for weights
def update(w, x, y, lr):
    # `lr` stands for "learning rate"
    grad = loss_gradient(w, x, y)
    updated_w = [w[j] - lr * grad[j] for j in range(d + 1)]
    return updated_w

## Stochastic gradient descent algorithm

In [7]:
def sgd(X, Y, d, initial_weights, lr):

    w = initial_weights
    
    while True:
        # Stochastic part of the algorithm: choose a data point at random for this iteration.
        i = random.randint(0, d)
        x, y = X[i], Y[i]

        # Update weights
        w = update(w, x, y, lr)

        # Hopefully, loss decreased compared to previous iteration
        current_loss = loss(w, X, Y)
        
        yield w, current_loss

## Run the algorithm

In [8]:
# Set learning rate
lr = 0.01

# Stop when loss is small enough
min_loss = 1e-8

# Alternatively, stop when a given number of steps is reached (in case the algorithm is diverging)
max_steps = 1000

# Initialize weights
weights = [0, 0]

# Initialize algorithm

current_loss = 1
step = 0

loss_history = []
weights_history = []

gen = sgd(X, Y, d, weights, lr)

# Run algorithm

while current_loss > min_loss and step < max_steps:

    weights, current_loss = next(gen)

    weights_history.append(weights)
    loss_history.append(current_loss)

    step += 1

    # Pretty-printing ironically requires code that is not so pretty
    if step % 10 ** (len(str(step)) - 1) == 0 or step == max_steps or current_loss < min_loss:
        print('step %s, loss = %.10f' % (str(step).zfill(len(str(max_steps)) - 1), current_loss))

step 001, loss = 23.2000000000
step 002, loss = 21.5296000000
step 003, loss = 21.5296000000
step 004, loss = 17.2236800000
step 005, loss = 19.9794688000
step 006, loss = 26.1524357120
step 007, loss = 33.6305441997
step 008, loss = 41.2384613630
step 009, loss = 48.3650705325
step 010, loss = 54.7321341489
step 020, loss = 10.0013254902
step 030, loss = 8.3115094863
step 040, loss = 4.6629849914
step 050, loss = 14.0476810412
step 060, loss = 2.9147519159
step 070, loss = 2.1467822271
step 080, loss = 1.3874731354
step 090, loss = 0.9200701256
step 100, loss = 0.8445847999
step 200, loss = 0.0100277604
step 300, loss = 0.0002815274
step 400, loss = 0.0000050402
step 500, loss = 0.0000001873
step 591, loss = 0.0000000087


In [9]:
# Final weights
weights

[-0.19999583014939817, 3.3999291125397684]

## Check that the model is actually right

In [10]:
for i in range(N):
    x, y = X[i], Y[i]
    print('f({x}) = {result} ≈ {y}'.format(x=X[i], result=f(weights, x), y=y))

f(7) = 1.9999583014939812 ≈ 2
f(-3) = 3.999916602987963 ≈ 4


## Plot loss history

In [11]:
x_plot = list(range(1, len(loss_history) + 1))
y_plot = list(loss_history)
data = [go.Scatter(x=x_plot, y=y_plot)]
layout = go.Layout(xaxis={'title': 'step'}, yaxis={'title': 'loss'})
figure = go.Figure(data = data, layout = layout)
plotly.iplot(figure)

## Plot weights history

In [12]:
x_plot = list(map(operator.itemgetter(0), weights_history))
y_plot = list(map(operator.itemgetter(1), weights_history))
data = [go.Scatter(x=x_plot, y=y_plot)]
layout = go.Layout(xaxis={'title': 'weight_x'}, yaxis={'title': 'weight_y'})
figure = go.Figure(data = data, layout = layout)
plotly.iplot(figure)

## References

https://en.wikipedia.org/wiki/Stochastic_gradient_descent

https://ml-cheatsheet.readthedocs.io/en/latest/gradient_descent.html